This is a vectorized version of the environment. How does it work? Simply put, instead of running a single environment, we're running them into batches (vectorization). At each time step, instead of formulating a single action $a$, we'll define it as a vector $a=[0, .., n_\text{envs}]$ where each entry corresponds to the action to be performed for an environment.

Thus, if you run 4 simulatenous environments, your observations space becomes (4, num_observations). Because we use a step size of 10 and have a total of 9 snesors, each observation will result into a space of (40,9) elements. The core idea is to speed up the inference and training time of the model instead of querying a single environment.

In our implementation, based on what model you decide, you should take as inputs the observations reshaped to your liking, and predict the actions $a$ from your policy.

In [ ]:
import numpy as np
from student_client.student_gym_env_vectorized import create_student_gym_env_vectorized

env = create_student_gym_env_vectorized(
            num_envs=4,
            step_size=10,
            user_token='your_token'
        )

In [ ]:
print(f"Environment created with {env.num_envs} parallel environments")
print(f"   Episode IDs: {env.episode_ids}")

# Reset all environments
print(f"\n🔄 Resetting all environments...")
observations, infos = env.reset()

print(f"   Observations shape: {observations.shape}")
print(f"   First observation: {observations[0]}")

## Training / Iterations

Here you can iterate through the vectorized environments. You'll notice that actions are a vector where each entry corresponds to the associated environment in the vector.

In A), we automatically reset the envs that have terminated so you can continue for an indefinite amount of steps. As environments don't have the same length, they stop at different times, this helps you reset terminated episodes on the fly.

Tips:
- The step_size return many observations, should you feed each one-by-one in your model, or the full step_size=10 one? The choice is yours!
- There exists multiple ways of exploring the dataset

In [ ]:
for step in range(40):

    # A) Check if any environments terminated
    terminated_envs = env.get_terminated_env_indices()
    if terminated_envs:
        print(f"   ⚠️  Environments {terminated_envs} terminated")
        reset_obs, reset_infos = env.reset_specific_envs(terminated_envs)
        for i, env_id in enumerate(terminated_envs):
            infos[env_id] = reset_infos[i] # reset previous info dict

    # Generate random actions for all environments
    actions = np.random.randint(0, 3, size=env.num_envs)

    print(f"\n   Step {step + 1}:")
    print(f"      Actions: {actions}")

    # Take step
    observations, rewards, terminateds, truncateds, infos = env.step(actions)

    print(f"      Rewards: {rewards}")
    print(f"      Terminated: {terminateds}")
    print(f"      Active environments: {env.get_active_count()}/{env.num_envs}")

    # Show filtered info (production mode)
    for i, info in enumerate(infos):
        print(f"      Env {i} info: {info}")

env.close()

In [ ]:
observations